# Conexión a base de datos Amazon Timestream usando Jupyter Notebook para análisis de datos de sensores
## IMF Smart Education - Master Business Analytics y Big Data - Trabajo de fin de master
### Autor: Luis Castillo


En este notebook se demuestra el procedimiento para extraer y procesar datos de una base de datos Amazon Timestream usando las librerías que pone a nuestra disposición AWS: *boto3* y *awswrangler*.  
* __boto3__: Librería que ofrece una API orientada a objetos que permite crear, configurar y gestionar servicios de AWS.
* __awswrangler__: Ofrece una capa de abstracción que permite importar datos de servicios de bases de datos de AWS en un dataframe Pandas.
 
 Tener en cuenta que el objetivo es simplemente demostrar el procedimiento de extracción de datos, no un análisis detallado de los mismos.  

 Iniciamos cargando las 3 librerías a utilizar:

In [8]:
# Carga de librerías
import awswrangler as wr
import boto3
import pandas as pd
import numpy as np

Con el siguiente código podemos extraer los datos de la base de datos Timestream y almacenarlos en un dataframe Pandas. Tener en cuenta que es necesario contar con credenciales configuradas en AWS-CLI.

In [9]:
# Definir algunas constantes
DB_NAME = 'OperationsDB-demo'
TABLE_NAME = 'machines'
PERIOD = '30d'

# Inicializar una sesión boto3. Requiere tener configurado AWS-CLI con credenciales para acceso a AWS. 
session=boto3.Session()

# Acceder a datos de timestream usando sentencia SQL y guardar en un dataframe Pandas
sql = f'SELECT * FROM "{DB_NAME}"."{TABLE_NAME}" WHERE time >= ago({PERIOD})'
df_timestream = wr.timestream.query(sql=sql, chunked=False, boto3_session=session)

En la variable `df` contamos con los datos para los últimos 30 días. 

In [10]:
df_timestream.head()

,machineId,sensorType,location,machineType,sensorModel,sensorId,measure_name,time,measure_value::double,measure_value::bigint
0,1,1,building1,compressor,dragino_lsn50,61089b67-940f-4e33-9ec4-c92d472ad215,Temperature_C,2022-09-07 13:30:02.077,67.3,NaN
1,2,vibration,building1,engine,rejeee_sl500us,bdeb778a-9afc-4298-adb4-25505a87d3aa,Acceleration_X_mG,2022-09-07 13:30:02.187,NaN,443.0
2,2,vibration,building1,engine,rejeee_sl500us,bdeb778a-9afc-4298-adb4-25505a87d3aa,Acceleration_X_mG,2022-09-07 13:35:02.000,NaN,406.0
3,2,vibration,building1,engine,rejeee_sl500us,bdeb778a-9afc-4298-adb4-25505a87d3aa,Acceleration_X_mG,2022-09-07 13:40:01.963,NaN,441.0
4,2,vibration,building1,engine,rejeee_sl500us,bdeb778a-9afc-4298-adb4-25505a87d3aa,Acceleration_X_mG,2022-09-07 13:45:02.035,NaN,443.0


Nótese que las medidas de temperatura y aceleración se encuentran una debajo de la otra, lo cual dificulta el análisis de datos con pandas. En la siguiente sección acondicionaremos el dataset para poder utilizarlo más fácilmente

#### Pre-procesamiento de los datos

Con el siguiente código se promedian los datos en períodos de 15 minutos, se agrupan por máquina y se presentan en un dataframe con una variable por cada columna.

In [11]:
def process_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # Copiamos el dataframe original
    df_p = df.copy() 
    # Combinar columnas measure_value::double y measure_value::bigint (son excluyentes entre sí)
    df_p['value'] = df_p['measure_value::double'].fillna(0) + df_p['measure_value::bigint'].fillna(0)
    # Eliminar columnas innecesarias
    df_p.drop(['sensorId', 'sensorType', 'sensorModel', 'measure_value::double', 'measure_value::bigint'], axis=1, inplace=True)
    # Redondeamos la fecha en períodos de 15 minutos 
    df_p['time'] = df_p['time'].dt.floor('15min')
    # Pivotar los datos 
    df_p = df_p.pivot_table(
        values='value',
        index=['time', 'machineId', 'location', 'machineType'],
        columns='measure_name',
        aggfunc=np.mean,
        fill_value=np.nan,
        ).reset_index()
    df_p.columns.name=None
    return df_p

In [12]:
df = process_dataframe(df_timestream)

Finalmente, tenemos el df listo para análisis:

In [13]:
df.head()

,time,machineId,location,machineType,Acceleration_X_mG,Acceleration_Y_mG,Acceleration_Z_mG,Temperature_C
0,2022-09-07 13:30:00,1,building1,compressor,643.333333,613.666667,708.000000,67.200000
1,2022-09-07 13:30:00,10,building2,engine,-257.333333,-224.333333,-280.333333,74.033333
2,2022-09-07 13:30:00,11,building2,pump,1197.333333,1212.000000,1208.333333,82.833333
3,2022-09-07 13:30:00,12,building2,motor,427.333333,482.666667,428.333333,64.866667
4,2022-09-07 13:30:00,13,building2,compressor,470.666667,463.666667,374.333333,79.033333


#### Análisis exploratorio de datos  

Se presenta un análisis básico de los datos disponibles en el dataframe. Recordar que los datos disponibles en la base de datos son simulados (no corresponden a sensores reales), lo que limita el análisis que se puede realizar

In [16]:
# Estadísticas básicas
df.describe()

,Acceleration_X_mG,Acceleration_Y_mG,Acceleration_Z_mG,Temperature_C
count,2420.000000,2420.000000,2420.000000,2420.000000
mean,55.023967,56.706061,54.845730,69.449402
std,561.706385,562.710487,563.017967,12.636887
min,-729.666667,-781.666667,-787.000000,50.733333
25%,-517.000000,-501.666667,-506.166667,64.900000
50%,21.666667,20.166667,26.666667,69.066667
75%,478.666667,483.666667,480.083333,76.966667
max,1252.000000,1296.333333,1310.000000,131.410000


In [19]:
# Verificar si hay nulos
df.isna().sum()

time                 0
machineId            0
location             0
machineType          0
Acceleration_X_mG    0
Acceleration_Y_mG    0
Acceleration_Z_mG    0
Temperature_C        0
dtype: int64

In [20]:
# Calcular promedio de temperatura y aceleración por tipo de máquina.
df.groupby('machineType').mean()

,Acceleration_X_mG,Acceleration_Y_mG,Acceleration_Z_mG,Temperature_C
machineType,,,,
compressor,442.101377,445.273829,443.385675,70.001267
engine,-119.325069,-119.549862,-121.131129,59.596033
motor,-99.769146,-96.353719,-99.185124,74.263444
pump,-2.911295,-2.546006,-3.686501,73.936865


In [24]:
# Encontrar las tres máquinas con mayor temperatura promedio
df.groupby('machineId').mean()['Temperature_C'].sort_values(ascending=False).head(3)


machineId
15    85.327576
16    85.312314
20    83.990854
Name: Temperature_C, dtype: float64

#### Conclusiones

En este notebook se muestra el procedimiento para acceder a los datos disponibles en Amazon Timestream usando las APIs provistas por Amazon. De esta forma, un analista o científico de datos puede apalancarse en el lenguaje Python para realizar tareas complejas de analítica descriptiva o predictiva.